In [14]:
import requests
import time
import random
import folium
import webbrowser
import pandas as pd
import openpyxl

def search_properties_by_condition(region_code, house_type, transaction_type):
    url = f"https://m.land.naver.com/cluster/ajax/articleList?rletTpCd={house_type}&tradTpCd={transaction_type}&cortarNo={region_code}&showR0&page=1"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://m.land.naver.com/map/"
    }

    all_data = []

    for attempt in range(100):
        try:
            print(f">> {url}")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            
            if not data.get('body'):
                print("더 이상 정보가 없습니다.")
                break
            
            # 현재 페이지의 데이터를 all_data 리스트에 추가
            for property_info in data['body']:
                house_info = {
                    '주택 유형': property_info.get('rletTpNm', ''),
                    '거래 유형': property_info.get('tradTpNm', ''),
                    '가격': int(str(property_info.get('prc', '')).replace(',', '')), # 가격을 문자열로 변환 후 쉼표 제거하여 정수로 변환
                    '면적(spc1)': property_info.get('spc1', ''),
                    '면적(spc2)': property_info.get('spc2', ''),
                    '방향': property_info.get('direction', ''),
                    '게시일': property_info.get('atclCfmYmd', ''),
                    '특징 설명': property_info.get('atclFetrDesc', ''),
                    '위도': property_info.get('lat', ''),
                    '경도': property_info.get('lng', ''),
                    '아파트명': property_info.get('atclNm', ''),  # 아파트명 추가
                    '동호수': property_info.get('bildNm', ''),  # 동호수 추가
                    '업체': property_info.get('rltrNm', '')     # 업체 추가
                    }
                all_data.append(house_info)
            
            # 추가 페이지를 요청하기 위해 URL 변경
            url_split = url.split("&page=")
            page_number = int(url_split[1])
            next_page_url = url_split[0] + "&page=" + str(page_number + 1)
            url = next_page_url
        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류 발생: {str(e)} (시도 {attempt + 1}/5)")
            time.sleep(2 ** attempt)
        except ValueError:
            print("JSON 디코딩 오류 발생")
            break
        retry_interval = random.uniform(1, 5)
        time.sleep(retry_interval)

    # 가격을 기준으로 오름차순으로 정렬
    all_data_sorted = sorted(all_data, key=lambda x: x['가격'])
    
    return all_data_sorted

def calculate_center_of_coordinates(coordinates):
    """
    좌표들의 평균값을 계산하여 반환합니다.
    :param coordinates: 좌표들의 리스트. 각 좌표는 [위도, 경도] 형태여야 합니다.
    :return: 평균 좌표 [평균 위도, 평균 경도]
    """
    total_lat = sum(coord[0] for coord in coordinates)
    total_lng = sum(coord[1] for coord in coordinates)
    avg_lat = total_lat / len(coordinates)
    avg_lng = total_lng / len(coordinates)
    return [avg_lat, avg_lng]

def display_properties_on_map(properties):
    # 주택의 모든 좌표를 리스트로 저장
    coordinates = [[property_info['위도'], property_info['경도']] for property_info in properties]

    # 주택 좌표들의 중심 좌표 계산
    center = calculate_center_of_coordinates(coordinates)

    # 중심 좌표로 초기화한 지도 객체 생성
    m = folium.Map(location=center, zoom_start=11)

    # 주택 정보를 지도에 추가
    for property_info in properties:
        lat = property_info['위도']
        lng = property_info['경도']
        description = f"주택 유형: {property_info['주택 유형']}<br>" \
                      f"거래 유형: {property_info['거래 유형']}<br>" \
                      f"가격: {property_info['가격']}<br>" \
                      f"면적(spc1): {property_info['면적(spc1)']}<br>" \
                      f"면적(spc2): {property_info['면적(spc2)']}<br>" \
                      f"방향: {property_info['방향']}<br>" \
                      f"게시일: {property_info['게시일']}<br>" \
                      f"특징 설명: {property_info['특징 설명']}"
        folium.Marker(
            location=[lat, lng],
            popup=description,
            icon=folium.Icon(color='blue', icon='home')
        ).add_to(m)

    m.save('properties_map.html')

def save_properties_to_excel(properties, file_name):
    # DataFrame 생성
    df = pd.DataFrame(properties)

    # 엑셀 파일로 저장
    df.to_excel(file_name, index=False)


if __name__ == "__main__":
    region_code = "1174010700"  # 예시: 암사동 법정동코드
    house_type = "APT"  # 예시: 주택 유형
    transaction_type = "A1"  # 예시: 거래 유형

    print(nL.find_address_codes("암사동", file_path="address_code.txt"))
    all_properties = search_properties_by_condition(region_code, house_type, transaction_type)

    display_properties_on_map(all_properties)
    webbrowser.open("properties_map.html")

    for property_info in all_properties:
        print(property_info)

    excel_file_name = "properties.xlsx"
    save_properties_to_excel(all_properties, excel_file_name)

{'서울특별시 강동구 암사동': '1174010700'}
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=1
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=2
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=3
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=4
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=5
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=6
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=7
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTpCd=A1&cortarNo=1174010700&showR0&page=8
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT&tradTp